In [19]:
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

# LLM Generator
from langchain_community.llms import CTransformers
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

config = {'max_new_tokens': 256,
          'temperature': 0.4,
          'repetition_penalty': 1.1,
          'context_length': 4096, # Set to max for Chat Summary, Llama-2 has a max context length of 4096
          }

llm = CTransformers(model='W:\\Projects\\LangChain\\models\\quantizedGGUF-theBloke\\llama-2-7b-chat.Q2_K.gguf',
                    callbacks=[StreamingStdOutCallbackHandler()],
                    config=config,
                    gpu_layers = 25)

# Implement another function to pass an array of PDFs / CSVs / Excels
from rag_pipeline import instantiate_rag
retriever = instantiate_rag()

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage

from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationTokenBufferMemory

# Docs:- https://python.langchain.com/docs/integrations/chat/llama2_chat
from langchain_experimental.chat_models import Llama2Chat

# Define system and user message templates
with open('.\\prompts\\system_message_template.txt', 'r') as file:
            system_message_template = file.read().replace('\n', '')

template_messages = [
    SystemMessage(content=system_message_template),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{text}"),
]
prompt_template = ChatPromptTemplate.from_messages(template_messages)

In [36]:
model = Llama2Chat(llm=llm)
memory = ConversationBufferMemory(llm=llm, memory_key="chat_history", return_messages=True, max_token_limit=10000)
chain = LLMChain(llm=model, prompt=prompt_template, memory=memory, verbose=False)

In [23]:
human_responses = ['Nothing logged yet']
ai_responses = ['Nothing logged yet']

# Decide wether to place this in streamlit.py
# or make a new post_process.py and import that to streamlit
def extract_dialogues(text):
    '''
    returns a two lists for human and ai dialogues,
    '''
    human_dialogues = []
    ai_dialogues = []
    lines = text.split('\n')

    # Iterate through each line
    for line in lines:
        # Remove leading and trailing whitespace
        line = line.strip()

        # Check if the line starts with 'Human:' or 'AI:'
        if line.startswith('Human:'):
            # Extract the text after 'Human:'
            human_dialogues.append(line[len('Human:'):].strip())
        elif line.startswith('AI:'):
            # Extract the text after 'AI:'
            ai_dialogues.append(line[len('AI:'):].strip())
    return human_dialogues, ai_dialogues

def llm_generation(question):
    global human_responses, ai_responses
    print(chain.invoke(input=question))
    human_responses, ai_responses = extract_dialogues(memory.buffer_as_str)
    return 'Generation complete'

def update_list():
    global human_responses, ai_responses
    human_responses, ai_responses = extract_dialogues(memory.buffer_as_str)
    return 'responses updated'  

def is_depressed():
    # Implement Classification
    all_user_inputs = ''.join(human_responses)
    from nlp_models import sentiment_class, pattern_classification, corelation_analysis
    is_depressed = sentiment_class(all_user_inputs)
    return 'Not so depressed' if is_depressed[0][1] > 0.5 else 'is_depressed'

In [43]:
llm_generation('I think i just solved a critical problem')

{'text': '  "Congratulations on solving that problem! It can be very relieving and empowering when we overcome challenges. How are you feeling now?"', 'chat_history': [HumanMessage(content='awesome, what do you think about todays day'), AIMessage(content='  "I\'m here to listen and offer support. Today can be tough for many reasons. How are you feeling about it?"'), HumanMessage(content='I think the day is great'), AIMessage(content='  "That\'s great to hear! Can you tell me more about what\'s making today great for you?"'), HumanMessage(content='I think i just solved a critical problem'), AIMessage(content='  "Congratulations on solving that problem! It can be very relieving and empowering when we overcome challenges. How are you feeling now?"')]}


'Human: awesome, what do you think about todays day\nAI:   "I\'m here to listen and offer support. Today can be tough for many reasons. How are you feeling about it?"\nHuman: I think the day is great\nAI:   "That\'s great to hear! Can you tell me more about what\'s making today great for you?"\nHuman: I think i just solved a critical problem\nAI:   "Congratulations on solving that problem! It can be very relieving and empowering when we overcome challenges. How are you feeling now?"'

In [44]:
memory.buffer_as_str

'Human: awesome, what do you think about todays day\nAI:   "I\'m here to listen and offer support. Today can be tough for many reasons. How are you feeling about it?"\nHuman: I think the day is great\nAI:   "That\'s great to hear! Can you tell me more about what\'s making today great for you?"\nHuman: I think i just solved a critical problem\nAI:   "Congratulations on solving that problem! It can be very relieving and empowering when we overcome challenges. How are you feeling now?"'

In [46]:
ai_responses[-1]

'"Congratulations on solving that problem! It can be very relieving and empowering when we overcome challenges. How are you feeling now?"'